In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import gspread

# Define the required scopes
scopes = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets"
]

# Authenticate using the credentials file
flow = InstalledAppFlow.from_client_secrets_file('google_credentials.json', scopes=scopes)
credentials = flow.run_local_server(port=0)

# Initialize Google Drive and Google Sheets API clients
service = build('sheets', 'v4', credentials=credentials)
gc = gspread.authorize(credentials)

# Replace these with your spreadsheet ID and the numeric sheet ID (the tab’s ID)
spreadsheet_id = '1ODbgVthmJkhykYf4CuFfRLilObS3KWipxy258NtGHDw'
sheet_id = 0

In [ ]:
requests = []

# ---------------------------
# 1. Set Data Validation (Dropdowns)
# ---------------------------

# Dropdown for Column A (A2:A101)
data_validation_colA = {
    "setDataValidation": {
        "range": {
            "sheetId": sheet_id,
            "startRowIndex": 1,          # Row 2 (skip header row 1)
            "endRowIndex": 100,          # Up to row 100; adjust as needed
            "startColumnIndex": 0,       # Column B (A=0, B=1)
            "endColumnIndex": 1
        },
        "rule": {
            "condition": {
                "type": "ONE_OF_LIST",
                "values": [
                    {"userEnteredValue": "yes"},
                    {"userEnteredValue": "no"},
                ]
            },
            "showCustomUi": True
        }
    }
}

# Dropdown for Column B (B2:B101)
data_validation_colB = {
    "setDataValidation": {
        "range": {
            "sheetId": sheet_id,
            "startRowIndex": 1,          # Row 2 (skip header row 1)
            "endRowIndex": 100,          # Up to row 100; adjust as needed
            "startColumnIndex": 1,       # Column B (A=0, B=1)
            "endColumnIndex": 2
        },
        "rule": {
            "condition": {
                "type": "ONE_OF_LIST",
                "values": [
                    {"userEnteredValue": "empirical"},
                    {"userEnteredValue": "conceptual"},
                    {"userEnteredValue": "review"}
                ]
            },
            "showCustomUi": True
        }
    }
}

requests.append(data_validation_colA)
requests.append(data_validation_colB)

In [ ]:
# ---------------------------
# 2. Add Conditional Formatting (Coloring based on dropdown value)
# ---------------------------
# For Column B: color the cell background based on its text value.

# For "Red" option in Column A
conditional_format_red = {
    "addConditionalFormatRule": {
        "rule": {
            "ranges": [{
                "sheetId": sheet_id,
                "startRowIndex": 1,
                "endRowIndex": 100,
                "startColumnIndex": 1,
                "endColumnIndex": 2
            }],
            "booleanRule": {
                "condition": {
                    "type": "TEXT_EQ",
                    "values": [{"userEnteredValue": "no"}]
                },
                "format": {
                    "backgroundColor": {
                        "red": 1.0,
                        "green": 0.8,
                        "blue": 0.8
                    }
                }
            }
        },
        "index": 0
    }
}

# For "Green" option in Column B
conditional_format_green = {
    "addConditionalFormatRule": {
        "rule": {
            "ranges": [{
                "sheetId": sheet_id,
                "startRowIndex": 1,
                "endRowIndex": 100,
                "startColumnIndex": 1,
                "endColumnIndex": 2
            }],
            "booleanRule": {
                "condition": {
                    "type": "TEXT_EQ",
                    "values": [{"userEnteredValue": "yes"}]
                },
                "format": {
                    "backgroundColor": {
                        "red": 0.8,
                        "green": 1.0,
                        "blue": 0.8
                    }
                }
            }
        },
        "index": 1
    }
}

# Similarly, you can add formatting for Column C. Here’s an example for "Small":
conditional_format_empirical = {
    "addConditionalFormatRule": {
        "rule": {
            "ranges": [{
                "sheetId": sheet_id,
                "startRowIndex": 1,
                "endRowIndex": 100,
                "startColumnIndex": 2,
                "endColumnIndex": 3
            }],
            "booleanRule": {
                "condition": {
                    "type": "TEXT_EQ",
                    "values": [{"userEnteredValue": "empirical"}]
                },
                "format": {
                    "backgroundColor": {
                        "red": 0.9,
                        "green": 0.9,
                        "blue": 0.7
                    }
                }
            }
        },
        "index": 0
    }
}

# For "Medium" in Column C
conditional_format_conceptual = {
    "addConditionalFormatRule": {
        "rule": {
            "ranges": [{
                "sheetId": sheet_id,
                "startRowIndex": 1,
                "endRowIndex": 100,
                "startColumnIndex": 2,
                "endColumnIndex": 3
            }],
            "booleanRule": {
                "condition": {
                    "type": "TEXT_EQ",
                    "values": [{"userEnteredValue": "conceptual"}]
                },
                "format": {
                    "backgroundColor": {
                        "red": 0.7,
                        "green": 0.9,
                        "blue": 0.7
                    }
                }
            }
        },
        "index": 1
    }
}

# For "Large" in Column C
conditional_format_review = {
    "addConditionalFormatRule": {
        "rule": {
            "ranges": [{
                "sheetId": sheet_id,
                "startRowIndex": 1,
                "endRowIndex": 100,
                "startColumnIndex": 2,
                "endColumnIndex": 3
            }],
            "booleanRule": {
                "condition": {
                    "type": "TEXT_EQ",
                    "values": [{"userEnteredValue": "review"}]
                },
                "format": {
                    "backgroundColor": {
                        "red": 0.7,
                        "green": 0.7,
                        "blue": 0.9
                    }
                }
            }
        },
        "index": 2
    }
}

# Add all conditional formatting requests to our list
requests.extend([
    conditional_format_red,
    conditional_format_green,
    conditional_format_empirical,
    conditional_format_conceptual,
    conditional_format_review
])

In [ ]:
requests

In [ ]:
# ---------------------------
# 3. Execute the Batch Update
# ---------------------------
batch_update_request_body = {
    'requests': requests
}

response = service.spreadsheets().batchUpdate(
    spreadsheetId=spreadsheet_id,
    body=batch_update_request_body
).execute()

print("Batch update response:", response)